In [ ]:
#we have created a model already
#now we have a inbulit face detector bmodel so just load these model here in this file

In [1]:
from tensorflow.keras.applications.mobilenet_v2 import preprocess_input
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras.models import load_model
from imutils.video import VideoStream
import numpy as np
import imutils
import time
import cv2
import os

In [2]:
#lets load the face detection mode it facenet
prototxtPath=r"C:\Users\dell\deploy.prototxt.txt"
weightsPath=r"C:\Users\dell\res10_300x300_ssd_iter_140000.caffemodel"
#dnn is deep neural network developed by cv2

faceNet=cv2.dnn.readNet(prototxtPath,weightsPath)

In [3]:
#load our mask detector model
maskNet=load_model("mask_detector.model")

Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


In [10]:
#method is called down
def detect_and_predict_mask(frame,faceNet,maskNet):
    
    #take the dimension of the frame and the costruct a blob from it
    (h,w)=frame.shape[:2]
    blob=cv2.dnn.blobFromImage(frame,1.0,(224,224),(104.0,177.0,123.0))
    
    #pass the blob through the network and obtain the face detections
    faceNet.setInput(blob)
    detection=faceNet.forward()
    print(detection.shape)
    
    #initialize our list of faces, their crrsp locations n the list of predictions
    #from our face mask network
    faces=[]
    locs=[]
    preds=[]
    
    #loop over the dections
    for i in range(0,detection.shape[2]):
        #extract the confidence i.e probability associated with the detection
        confidence=detection[0,0,i,2]
        
        #filter out weak detection by ensuring the confidence is greater 
        #than the minimum confidence
        if confidence>0.5:
            #compute the(x,y) - coordinates of the bounding box for the object
            box=detection[0,0,i,3:7]*np.array([w,h,w,h])
            (startX,startY,endX,endY)=box.astype('int')
            
            #ensure the bounding boxes fall within the dimensions of the frame
            (startX,startY)=(max(0,startX),max(0,startY))
            (endX,endY)=(min(w-1,endX),min(h-1,endY))
            
            #Exctract the face ROI convert it from BGR to RGB channel
            #ordering,resize it to 224*224, and preprocess it
            face=frame[startY:endY,startX:endX]
            face=cv2.cvtColor(face,cv2.COLOR_BGR2RGB)
            face=cv2.resize(face,(224,224))
            face=img_to_array(face)
            face=preprocess_input(face)
            
            #add the face and bounding boxes to their respective list
            faces.append(face)
            locs.append((startX,startY,endX,endY))
        
    #only make a prediction if at least one face is detected
    if len(faces)>0:
        #for faster inference lets use the batch prediction on 'all' faces
        #at the same time rather than one by one pred in the above loop
        faces=np.array(faces,dtype="float32")
        preds=maskNet.predict(faces,batch_size=32)
    
    #return the 2 tuples of the dace location and corrsp predictions
    return(locs,preds)  
            



#initialize the vdo stream
vs=VideoStream(src=0).start()# src=0 means primary camara its pc's

# loop over the frames from the vdo stream
while True:
    #grab the fram from the threaded vdo stream and 
    #resize it to the max width of 440px
    frame=vs.read()
    frame=imutils.resize(frame,width=400)
    
    #now to detect the face in the each frame and detect whether they are wearing the mask or not
    #lets call a method which returns two values,
    #1.locs: (x,y) coordinates of the faces within the rectangualr box
    #2.preds: accuracy of wearing mask or without mask
    
    (locs,preds)=detect_and_predict_mask(frame,faceNet,maskNet)
    
    #loop over the detected face locations n predictions
    for (box,pred) in zip(locs,preds):
        #unpack the bounding box and predictions
        (startX,startY,endX,endY)=box
        (mask,withoutMask)=pred
        
        #determine the class label and color we'll use 
        #to draw the bounding box and texts
        label="MASK" if mask>withoutMask else "NO MASK!!"
        color=(0,255,0) if label=="MASK" else (0,0,255) # (B,G,R)
        #to display the percentage of prediction
        label="{}:{:.2f}%".format(label,max(mask,withoutMask)*100) 
        
        #display the probalibilty in the label and bounding box rectangle
        #on the op frame
        cv2.putText(frame,label,(startX,startY-10),
                    cv2.FONT_HERSHEY_SIMPLEX,0.45,color,2) # startY-10 cz let the text appear bit above the box 
        cv2.rectangle(frame,(startX,startY),(endX,endY),color,2)
    #show the op frame
    cv2.imshow("Frame",frame)
    key=cv2.waitKey(1) & 0xFF
    
    #break the loop for 'q'
    if key==ord("q"):
        break
cv2.destroyAllWindows()
vs.stop()
    

(1, 1, 200, 7)
(1, 1, 200, 7)
(1, 1, 200, 7)
(1, 1, 200, 7)
(1, 1, 200, 7)
(1, 1, 200, 7)
(1, 1, 200, 7)
(1, 1, 200, 7)
(1, 1, 200, 7)
(1, 1, 200, 7)
(1, 1, 200, 7)
(1, 1, 200, 7)
(1, 1, 200, 7)
(1, 1, 200, 7)
(1, 1, 200, 7)
(1, 1, 200, 7)
(1, 1, 200, 7)
(1, 1, 200, 7)
(1, 1, 200, 7)
(1, 1, 200, 7)
(1, 1, 200, 7)
(1, 1, 200, 7)
(1, 1, 200, 7)
(1, 1, 200, 7)
(1, 1, 200, 7)
(1, 1, 200, 7)
(1, 1, 200, 7)
(1, 1, 200, 7)
(1, 1, 200, 7)
(1, 1, 200, 7)
(1, 1, 200, 7)
(1, 1, 200, 7)
(1, 1, 200, 7)
(1, 1, 200, 7)
(1, 1, 200, 7)
(1, 1, 200, 7)
(1, 1, 200, 7)
(1, 1, 200, 7)
(1, 1, 200, 7)
(1, 1, 200, 7)
(1, 1, 200, 7)
(1, 1, 200, 7)
(1, 1, 200, 7)
(1, 1, 200, 7)
(1, 1, 200, 7)
(1, 1, 200, 7)
(1, 1, 200, 7)
(1, 1, 200, 7)
(1, 1, 200, 7)
(1, 1, 200, 7)
(1, 1, 200, 7)
(1, 1, 200, 7)
(1, 1, 200, 7)
(1, 1, 200, 7)
(1, 1, 200, 7)
(1, 1, 200, 7)
(1, 1, 200, 7)
(1, 1, 200, 7)
(1, 1, 200, 7)
(1, 1, 200, 7)
(1, 1, 200, 7)
(1, 1, 200, 7)
(1, 1, 200, 7)
(1, 1, 200, 7)
(1, 1, 200, 7)
(1, 1, 200, 7)
(1, 1, 200